<a href="https://colab.research.google.com/github/kinjaljoshi/face_sentiment_analysis/blob/main/face_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

--2024-06-29 21:49:25--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  94.4MB/s    in 0.6s    

2024-06-29 21:49:26 (94.4 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
!pip install opencv-python dlib imutils tensorflow keras

In [ ]:
import cv2
import dlib
from imutils import face_utils
from google.colab.patches import cv2_imshow

# Load facial landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

def highlight_facial_features(frame, gray):
    faces = detector(gray)
    for face in faces:
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        # Draw facial landmarks
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

    return frame

def main():
    # Open webcam
#    cap = cv2.VideoCapture('/content/WIN_20240629_18_59_15_Pro.mp4')
    cap = cv2.VideoCapture('/content/mixed.mp4')
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Extract and highlight facial features
        frame = highlight_facial_features(frame, gray)

        # Display the processed frame
        #cv2.imshow('Facial Features Highlighted', frame)
        cv2_imshow(frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Path to the video file
video_path = 'path_to_your_video.mp4'

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Read until the end of the video
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If frame is read correctly, ret is True
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break

    # Convert the frame to grayscale (face detection works better on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the resulting frame
    cv2_imshow(frame)

    # Wait for a short period to control the frame rate
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()

In [ ]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from keras.models import load_model
from google.colab.patches import cv2_imshow

# Load pre-trained emotion classification model
emotion_model_path = '/content/emotion_model.hdf5'
emotion_model = load_model(emotion_model_path)

# Load facial landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def extract_face_features(frame, gray):
    faces = detector(gray)
    for face in faces:
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        # Draw facial landmarks
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

        # Extract the region of interest (ROI) for emotion classification
        (x, y, w, h) = face_utils.rect_to_bb(face)
        roi = gray[y:y + h, x:x + w]
        roi = cv2.resize(roi, (64, 64))
        roi = roi.astype('float') / 255.0
        roi = np.expand_dims(roi, axis=0)
        roi = np.expand_dims(roi, axis=-1)

        # Predict emotion
        emotion_prediction = emotion_model.predict(roi)[0]
        emotion_label = emotion_labels[np.argmax(emotion_prediction)]

        # Display the emotion label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return frame

def main():
    # Open webcam
    cap = cv2.VideoCapture('/content/test-2.mp4')

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Extract facial features and classify emotions
        frame = extract_face_features(frame, gray)

        # Display the processed frame
        cv2_imshow(frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
